In [1]:
import pandas as pd
import numpy as np

In [2]:
#Step 1: Import document, Change Program Start Date to Date of Identification
df = pd.read_csv("BFZ_June.csv",parse_dates=['Program Start Date','Program End Date'])
df = df.rename(columns={"Program Start Date":"Date of Identification",'Case Number':'Client ID','Veteran Status (HUD)':'Veteran Status'})
df = df.drop('Name',axis=1)
df = df.drop_duplicates()
df = df.reset_index()
df = df.drop(['index'],axis=1)

In [3]:
#Step 2: Adding Household Type Column
household_type = {"Arlington Zero: Chronic - Veterans Only":"Single Adults",
"Arlington Zero: Single Adults":"Single Adults",
"Arlington Zero: Families":"Family",
"Arlington Zero: TAY":"Youth"}
df['Household Type'] = df['Program Name'].map(household_type)
df['Household Type'].value_counts()

Single Adults    447
Family            42
Youth              6
Name: Household Type, dtype: int64

In [4]:
#Step 3: Adding Client ID Counter and Client ID Household Counter
#Client ID Counter
df['Client ID Counter'] = df['Client ID'].map(df.groupby('Client ID').agg({'Client ID':'count'})['Client ID'])
#Client ID Household Counter
counter = {}
for i in df['Client ID']:
    if i not in counter:
        counter[i] = {"Single Adults":0,"Family":0,"Youth":0}
for j in counter:
    for k in df[df['Client ID']==j]['Household Type']:
        counter[j][k] += 1
df['Client ID Household Counter'] = np.nan
for i in df.index:
    df.loc[i,'Client ID Household Counter'] = counter[df.loc[i,'Client ID']][df.loc[i,'Household Type']]
df['Client ID Household Counter'] = df['Client ID Household Counter'].apply(int)
df['Client ID Household Counter'].value_counts()

1    398
2     78
3     15
4      4
Name: Client ID Household Counter, dtype: int64

In [5]:
#Step 4: Adding Chronic Column
#If client is in Chronic & Vet and 'No' to Veteran Status, then they are chronic
df['Chronic Status'] = np.nan
df.loc[(df['Program Name']=="Arlington Zero: Chronic - Veterans Only") & (df['Veteran Status']=="No")
,"Chronic Status"] = "Yes"
df['Chronic Status'].value_counts()

Yes    233
Name: Chronic Status, dtype: int64

In [6]:
#Step 5, remap all dismissal reasons
from values import dismissal
df['Dismissal Reason'] = df['Dismissal Reason'].map(dismissal)
df['Dismissal Reason'].value_counts()

Housed                                 290
No longer meets population criteria     55
Inactive                                50
Name: Dismissal Reason, dtype: int64

In [7]:
#Step 6, Populate Housing Move-In Date
df['Housing Move-In Date'] = df["Program End Date"][df["Dismissal Reason"]=="Housed"]
df.head()

,Age,Client ID,Family Name,Relationship,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date
0,57.0,38049,DUANE SMITH Family,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,No,2021-02-15,2021-03-24,Housed,Single Adults,1,1,Yes,2021-03-24
1,62.0,9330,John Doherty Family,Self/Head of Household,Arlington Zero: Single Adults,No,2019-07-01,2020-06-30,NaN,Single Adults,1,1,NaN,NaT
2,58.0,7273,Bowman Family 826,Non-married Partner,Arlington Zero: Chronic - Veterans Only,No,2015-09-25,2016-06-29,No longer meets population criteria,Single Adults,2,2,Yes,NaT
3,27.0,4180,Bowman Family 329,Child,Arlington Zero: Chronic - Veterans Only,No,2016-02-04,2016-09-23,Housed,Single Adults,1,1,Yes,2016-09-23
4,59.0,3555,Bowman Family 315,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,No,2015-11-02,2017-03-13,Housed,Single Adults,1,1,Yes,2017-03-13


In [8]:
#Step 7, Populate Inactive Date
#Do we consider those with program end date and null dismissal reasons as inactive?
df['Inactive Date'] = df['Program End Date'][df["Dismissal Reason"]=="Inactive"]
df.head()

,Age,Client ID,Family Name,Relationship,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date,Inactive Date
0,57.0,38049,DUANE SMITH Family,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,No,2021-02-15,2021-03-24,Housed,Single Adults,1,1,Yes,2021-03-24,NaT
1,62.0,9330,John Doherty Family,Self/Head of Household,Arlington Zero: Single Adults,No,2019-07-01,2020-06-30,NaN,Single Adults,1,1,NaN,NaT,NaT
2,58.0,7273,Bowman Family 826,Non-married Partner,Arlington Zero: Chronic - Veterans Only,No,2015-09-25,2016-06-29,No longer meets population criteria,Single Adults,2,2,Yes,NaT,NaT
3,27.0,4180,Bowman Family 329,Child,Arlington Zero: Chronic - Veterans Only,No,2016-02-04,2016-09-23,Housed,Single Adults,1,1,Yes,2016-09-23,NaT
4,59.0,3555,Bowman Family 315,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,No,2015-11-02,2017-03-13,Housed,Single Adults,1,1,Yes,2017-03-13,NaT


In [9]:
#Step 8, Calculate 1stDateofID, then calculate Returned to Active Date (Date of Idenfication on second record)
#{FIXED [ClientID],[Household Type]: MIN([Date of Identification]) }
#IF [NumRows_ClientID_HouseholdType]> 1 AND [1stDateofID] <> [Date of Identification] THEN [Date of Identification] ELSE NULL END
#If [Household Type Case Counter]>1 AND [1stDateofID] != [Date of Identification] THEN [Date of Identification] ELSE np.nan
df['1stDateofID'] = df['Client ID'].map(df.groupby('Client ID').agg({'Date of Identification':'min'})['Date of Identification'])
df['Return to Active Date'] = np.nan
df.loc[(df['Client ID Household Counter']>1) & (df['1stDateofID']!=df['Date of Identification'])
,"Return to Active Date"] = df['Date of Identification']
df['Return to Active Date'] = pd.to_datetime(df['Return to Active Date'])

In [10]:
#Step 9, Calculate most recent move-in or inactive dates
df['Most Recent Move-In Date'] = df['Client ID'].map(df.groupby('Client ID').agg({'Housing Move-In Date':'max'})['Housing Move-In Date'])
df['Most Recent Inactive Date'] = df['Client ID'].map(df.groupby('Client ID').agg({'Inactive Date':'max'})['Inactive Date'])

In [11]:
#Step 10, Narrow down dataframe down to active clients and newly exited clients for the reporting month
dates = {}
dates["Reporting Year"] = input("Reporting Year? Enter four digits: ")
dates["Reporting Month"] = input("Reporting Month? Enter either proper string or number ")
dates['Start Date'] = pd.to_datetime(dates["Reporting Year"]+dates["Reporting Month"],format='%Y%m',errors='ignore')
dates['Last Day'] = dates['Start Date'].days_in_month
dates['Reporting Date'] = dates['Start Date'].replace(day=dates['Last Day'])
# dates['Reporting Date'] = pd.Timestamp('{0}-{1}-{2}'.format(dates['Reporting Date'].year,dates['Reporting Date'].month,dates['Last Day']))
# dates['Start Date'] = '{0}-{1}-1'.format(dates['Reporting Date'].year,dates['Reporting Date'].month)
# dates['Start Date'] = pd.Timestamp(dates['Start Date'])
print("Start Date {0}\nReporting Date {1}".format(dates['Start Date'],dates['Reporting Date']))

Start Date 2022-06-01 00:00:00
Reporting Date 2022-06-30 00:00:00


In [12]:
active_df = df[(df['Date of Identification']<=dates['Reporting Date']) &
((df['Program End Date'].isnull()==True) | (df['Program End Date']<df['Date of Identification']) | (df['Program End Date']>dates['Reporting Date']))]
exited_df = df[(df['Program End Date']>=dates['Start Date']) & (df['Program End Date']<=dates['Reporting Date'])]
filtered_df = pd.concat([active_df,exited_df])
filtered_df = filtered_df.reset_index()
filtered_df = filtered_df.drop(['index'],axis=1)
filtered_df.head()

,Age,Client ID,Family Name,Relationship,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date,Inactive Date,1stDateofID,Return to Active Date,Most Recent Move-In Date,Most Recent Inactive Date
0,74.0,5867,Alfred Taylor Family,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,Yes,2022-04-27,NaT,NaN,Single Adults,2,2,NaN,NaT,NaT,2015-01-29,2022-04-27,2015-08-17,NaT
1,63.0,35825,Phillip H. Cannon Family,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,No,2022-03-18,NaT,NaN,Single Adults,3,3,Yes,NaT,NaT,2015-10-28,2022-03-18,NaT,2016-06-15
2,58.0,51829,Taylor Travis Worthington Family,Self/Head of Household,Arlington Zero: Chronic - Veterans Only,Yes,2021-12-10,NaT,NaN,Single Adults,2,2,NaN,NaT,NaT,2020-03-17,2021-12-10,2020-11-19,NaT
3,53.0,44879,Zaitoon Mehrabi Family,NaN,Arlington Zero: Single Adults,No,2022-01-19,NaT,NaN,Single Adults,1,1,NaN,NaT,NaT,2022-01-19,NaT,NaT,NaT
4,75.0,5619,NaN,NaN,Arlington Zero: Chronic - Veterans Only,Client Refused,2020-04-02,NaT,NaN,Single Adults,2,2,NaN,NaT,NaT,2015-01-29,2020-04-02,NaT,2016-06-15


In [13]:
#Step 11, Determine clients that "No longer meets population criteria" by demographic info
#All persons, all singles, veterans, chronic, chronic veteran, youth, families
print("How many clients this month No longer meet population criteria?")
print("All clients ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")).count())
print("Singles ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")&(exited_df['Household Type']=='Single Adults')).count())
print("Veterans ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")&(exited_df['Veteran Status']=='Yes')).count())
print("Chronic ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")&(exited_df['Chronic Status']=='Yes')).count())
print("Chronic Veterans ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")&(exited_df['Chronic Status']=='Yes')&(exited_df['Veteran Status']=='Yes')).count())
print("Youth ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")&(exited_df['Household Type']=='Youth')).count())
print("Families ",
exited_df['Dismissal Reason'].where((exited_df['Dismissal Reason']=="No longer meets population criteria")&(exited_df['Household Type']=='Families')).count())

How many clients this month No longer meet population criteria?
All clients  3
Singles  3
Veterans  0
Chronic  1
Chronic Veterans  0
Youth  0
Families  0


In [14]:
#Step 12, Calculate BFZ Reporting Metrics to make sure numbers match
print("Actively Homeless ", len(active_df),
"+",exited_df['Client ID'].where(exited_df['Dismissal Reason']=='No longer meets population criteria').count(),'No longer meet population criteria')
print("Housing Placements ", exited_df['Client ID'].where(exited_df['Dismissal Reason']=='Housed').count())
print("Moved to Inactive ", exited_df['Client ID'].where(exited_df['Dismissal Reason']=='Inactive').count())
print("Newly Identified Inflow ",len(filtered_df.loc[(dates['Start Date']<=filtered_df['Date of Identification']) & (filtered_df['Date of Identification']<=dates['Reporting Date'])]))
def housing_lot():
    housed = exited_df[exited_df['Dismissal Reason']=='Housed']
    lot = [(housed.loc[i,'Program End Date'] - housed.loc[i,'Date of Identification']) for i in housed.index]
    average = 0
    for i in lot:
        average += i.days
    return average/len(lot)
print("Average Length of Time from ID to Housing Placement ",housing_lot())
def rtad_counter():
    rtad = active_df[active_df['Return to Active Date'].isnull()==False]
    counter = 0
    for i in rtad.index:
        if (rtad.loc[i,"Return to Active Date"].year==dates['Reporting Date'].year)& (rtad.loc[i,"Return to Active Date"].month==dates['Reporting Date'].month):
            counter += 1
    return counter
print("Returned to Active ",rtad_counter())
print("Number of children ",active_df['Relationship'].value_counts()['Child'])
print("Number of families ",active_df['Family Name'].where(active_df['Household Type']=='Family').nunique())

Actively Homeless  33 + 3 No longer meet population criteria
Housing Placements  2
Moved to Inactive  0
Newly Identified Inflow  6
Average Length of Time from ID to Housing Placement  83.5
Returned to Active  0
Number of children  1
Number of families  2


In [19]:
active_df['Relationship'].value_counts()['Child']

1

In [20]:
active_df['Family Name'].where(active_df['Household Type']=='Family').nunique()

2